In [1]:
import os
from tqdm import tqdm

import cv2
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from utils.dataloader import DataLoader
from utils.vis import MatplotlibVisualizer
from utils.transforms import HairRemoval, Composer
from descriptors.texture import LBPDescriptor, GLCMDescriptor
from utils.segmentation import ThresholdingSegmentation
from descriptors.bow import MultifeatureBoW
from descriptors.color import ColorDescriptor, ColorLayoutDescriptor, ColorCooccurrenceMatrixDescriptor

In [2]:
## Classes
CLASSES = ['nevus', 'others']

## Work folfer
work_folder = os.getcwd()
data_folder = os.path.join(work_folder, '..', 'Data/Challenge1/subdata')

## Visualizer
matplotlib_visualizer = MatplotlibVisualizer()

In [3]:
# color_descriptor = ColorDescriptor(bins=(8, 12, 3))
# color_cooccurrence_matrix_descriptor = ColorCooccurrenceMatrixDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8)
# glcm_descriptor = GLCMDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8, visualize=False, grid_x=4, grid_y=4)
lbp_descriptor = LBPDescriptor(radius=3, n_points=16, grid_x=3, grid_y=3, visualize=False)

descriptors = [lbp_descriptor]
classifier = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, learning_rate=0.1, n_jobs=-1)

# Initialize the MultifeatureBoW model
multifeature_bow = MultifeatureBoW(vocab_size=10, descriptors=descriptors)

In [21]:
train_dataloader = DataLoader(data_folder, 'train', 
                        shuffle=True, 
                        ignore_folders=['black_background', '.DS_Store'], 
                        max_samples=20, 
                        balance=False,
                        transforms=None,
                        classes=CLASSES,
                        mask=False)

# Build the codebook using the training data
multifeature_bow.build_codebook(train_dataloader)


# Train the classifier using BoW histograms
multifeature_bow.fit_classifier(train_dataloader)

Extracting Features for LBPDescriptor: 100%|██████████| 20/20 [00:01<00:00, 14.90it/s]


Fitting classifier...


Transforming Images: 100%|██████████| 20/20 [00:01<00:00, 15.54it/s]


In [22]:
test_dataloader = DataLoader(data_folder, 'val', 
                        shuffle=True, 
                        ignore_folders=['black_background', '.DS_Store'], 
                        max_samples=100, 
                        balance=False,
                        transforms=None,
                        classes=CLASSES, 
                        mask=False)

report, confusion_m = multifeature_bow.evaluate(test_dataloader, class_names=CLASSES)
print(report)
print(confusion_m)

Evaluating...
Predicting...


Transforming Images: 100%|██████████| 100/100 [00:08<00:00, 11.63it/s]


              precision    recall  f1-score   support

       nevus       0.77      0.66      0.71        56
      others       0.63      0.75      0.69        44

    accuracy                           0.70       100
   macro avg       0.70      0.71      0.70       100
weighted avg       0.71      0.70      0.70       100

[[37 19]
 [11 33]]
